In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ColabNotebooks/')

In [3]:
%pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 7.2 MB/s eta 0:00:00


In [4]:
import torch
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy, BinarySpecificity, BinaryPrecision, BinaryRecall
from model.smooth_cross_entropy import smooth_crossentropy
from coronary import Coronary
from utility.log import Log
from utility.initialize import initialize
from utility.step_lr import StepLR
from optimizers_impl.sam.sam import SAM
from utility.bypass_bn import enable_running_stats, disable_running_stats
from optimizers_impl.gsam import GSAM, CosineScheduler, ProportionScheduler

In [5]:
def train(model, device, train_loader, optimizer, log, scheduler):
    model.train()
    log.train(len_dataset=len(dataset.train))

    for batch in train_loader:
        inputs, targets = (b.to(device) for b in batch)
        inputs = inputs.permute(0, 3, 2, 1)

        predictions = model(inputs)
        loss = smooth_crossentropy(predictions.logits, targets, smoothing=0.0001)
        loss.mean().backward()
        optimizer.first_step(zero_grad=True)

        disable_running_stats(model)
        smooth_crossentropy(model(inputs).logits, targets, smoothing=0.0001).mean().backward()
        optimizer.second_step(zero_grad=True)

        with torch.no_grad():
            correct = torch.argmax(predictions.logits, 1) == targets
            log(model, loss.cpu(), correct.cpu(), scheduler.lr())
            # scheduler(epoch)
            scheduler.step()


In [6]:
def test(model, device, test_loader, log):
    model.eval()
    log.eval(len_dataset=len(test_loader))

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = (b.to(device) for b in batch)
            inputs = inputs.permute(0, 3, 2, 1)

            predictions = model(inputs)
            loss = smooth_crossentropy(predictions, targets, smoothing=0.0001)
            correct = torch.argmax(predictions, 1) == targets
            log(model, loss.cpu(), correct.cpu())

In [7]:
if __name__ == "__main__":

  num_epochs = 50
  # model_name_path = '/content/drive/MyDrive/ColabNotebooks/results_logs/'
  model_name = 'inceptionv3-sam'
  initialize(seed=111)
  log = Log(log_each=1, log_name=model_name)

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  dataset = Coronary(batch_size=100, threads=2, img_size_definition=299)
  log = Log(log_each=1, log_name=model_name)

  model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True).to(device)
  model.num_classes = 2
  model.eval()
  base_optimizer = torch.optim.SGD

  optimizer = SAM(model.parameters(), base_optimizer, rho=0.5, adaptive=True, lr=0.005, weight_decay=0.000001)

  scheduler = CosineScheduler(T_max=50, max_value=0.003, min_value=0.0, optimizer=optimizer)

  for epoch in range(num_epochs):
      train(model, device, dataset.train, optimizer, log, scheduler)
      test(model, device, dataset.valid, log)

  log.flush()


100
300
880


/content/drive/MyDrive/ColabNotebooks/coronary.py:86: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_tensor_x = torch.Tensor(train_xs)
Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights

┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨
┃           0  ┃      5.8830  │      0.06 %  ┃   2.857e-03  │       01:04  ┃      5.3295  │      0.67 %  ┃
┃           1  ┃      3.2717  │      0.54 %  ┃   2.304e-03  │       00:22  ┃      4.0983  │     24.67 %  ┃
┃           2  ┃      1.7447  │      0.82 %  ┃   1.500e-03  │       00:22  ┃      3.5086  │     37.67 %  ┃
┃           3  ┃      1.2162  │      0.84 %  ┃   6.963e-04  │       00:23  ┃      3.1319  │     45.67 %  ┃
┃           4  ┃      0.9771  │      0.87 %  ┃   1.428e-04  │       00:23  ┃      3.0566  │     49.33 %  ┃
┃           5  ┃      0.9392  │      

In [8]:
# if __name__ == "__main__":

#   num_epochs = 50
#   model_name_path = '/content/drive/MyDrive/ColabNotebooks/results_logs/'
#   model_name = model_name_path+'inceptionv3-sam'
#   initialize(seed=111)
#   log = Log(log_each=1, log_name=model_name)

#   device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#   dataset = Coronary(batch_size=100, threads=2, img_size_definition=299)
#   log = Log(log_each=1, log_name=model_name)

#   model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True).to(device)
#   model.num_classes = 2
#   model.eval()
#   base_optimizer = torch.optim.Adagrad

#   optimizer = SAM(model.parameters(), base_optimizer, rho=2.0, adaptive=True, lr=0.0007, weight_decay=0.1)

#   scheduler = CosineScheduler(T_max=50, max_value=0.0005, min_value=0.0, optimizer=optimizer)

#   for epoch in range(num_epochs):
#       train(model, device, dataset.train, optimizer, log, scheduler)
#       test(model, device, dataset.valid, log)

#   log.flush()


In [9]:
metricF1 = BinaryF1Score().to(device)
metricAcc = BinaryAccuracy().to(device)
metricSpecificity = BinarySpecificity().to(device)
metricPrecision = BinaryPrecision().to(device)
metricRecall = BinaryRecall().to(device)

In [11]:
predictions

tensor([[ 4.8691,  5.0991, -0.1687,  ..., -0.4300,  0.2951,  1.6680],
        [ 4.8473,  5.0123, -0.7797,  ..., -0.4765, -0.7518,  1.4696],
        [ 4.8709,  4.7498, -0.4697,  ..., -0.8655, -0.6485,  0.5697],
        ...,
        [ 5.9933,  4.5656, -0.4723,  ..., -1.2310, -0.8783,  1.5864],
        [ 4.7869,  5.3679,  0.0439,  ..., -0.7839, -0.0573,  1.5895],
        [ 5.8366,  4.8371, -0.3318,  ..., -0.7589, -0.4777,  0.9959]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [12]:
predictions[predictions != 0] = 1


In [17]:
predictions

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0',
       grad_fn=<IndexPutBackward0>)

In [15]:
inputs2 = dataset.test_x.permute(0, 3, 2, 1).to(device)

In [16]:
predictions2 = model(inputs2)

OutOfMemoryError: ignored

In [19]:
# inputs = dataset.test_x.permute(0, 3, 2, 1).to(device)
# predictions = model(inputs)
f1 = metricF1(predictions, dataset.test_y.to(device))
acc = metricAcc(torch.argmax(predictions, 1), dataset.test_y.to(device))
specificity = metricSpecificity(torch.argmax(predictions, 1), dataset.test_y.to(device))
precision = metricPrecision(torch.argmax(predictions, 1), dataset.test_y.to(device))
recall = metricRecall(torch.argmax(predictions, 1), dataset.test_y.to(device))

with open(model_name+'.txt', 'a') as file:
                file.write(f"\n \n F1: {f1}\n Accuracy: {acc}\n Specificity:{specificity}\n Precision: {precision}\n Recall: {recall}")

print(f1)
print(acc)
print(precision)
print(specificity)
print(recall)


RuntimeError: ignored